# Exploring the set of human k-mers

In [ ]:
import os, sys
import time
import json
import numpy as np
import matplotlib.pyplot as plt
from numpy.typing import NDArray
from Bio import SeqIO
import tqdm.notebook as tqdm

from humprot.helpers import sym2int
from humprot.helpers import get_kmers_from_sequences
from humprot.helpers import count_kmers_in_seqs, build_i2s_s2i_maps
from humprot.trees import MultiKmerTree
from humprot.constants import AA_LIST_STD20

In [ ]:
DATDIR = "../data"
OUTDIR = "../out/human_kmer_exploration"
IMGDIR = os.path.join(OUTDIR, "images")

os.makedirs(OUTDIR, exist_ok=True)
os.makedirs(IMGDIR, exist_ok=True)

human_mktree_fpath = os.path.join(DATDIR, "human_multikmer_trees", "human_noXU_mktree_25.npz")
human_fasta_fpath = os.path.join(DATDIR, "human_proteins_noXU.faa")

KMAX = 25
KS_ARR = np.arange(1, KMAX + 1)

if not os.path.isfile(human_mktree_fpath):
    raise FileNotFoundError(human_mktree_fpath)

if not os.path.isfile(human_fasta_fpath):
    raise FileNotFoundError(human_fasta_fpath)


In [ ]:
AA_LIST = AA_LIST_STD20
MASK = len(AA_LIST)
NUM_AAS = len(AA_LIST)

int2sym_map, sym2int_map = build_i2s_s2i_maps(AA_LIST, mask=MASK)

In [ ]:
tree = MultiKmerTree.load(human_mktree_fpath)
print(f"Loaded tree from file: {human_mktree_fpath}")
tree

In [ ]:
layer_sizes = tree.get_layer_sizes()
layer_totals = tree.get_layer_value_sums()

print("layer sizes:", layer_sizes)
print("layer totals:", layer_totals)


In [ ]:
# Plot number of unique realized k-mers as a function of k
fig, ax = plt.subplots(1, 1)

ax.semilogy(
    KS_ARR, layer_sizes, ".-",
)

ax.set_xlabel(f"$k$")
ax.set_ylabel("count")
ax.set_title("Number of unique realized human $k$-mers")

ax.grid(which='major', color='gray', linestyle='-', alpha=0.6)
ax.set_xlim(0, KMAX + 1)

plt.savefig(f"{IMGDIR}/k_vs_num_unique_kmers.png", bbox_inches=None)
plt.show()

In [ ]:
# Plot coverage of kmer space as a function of k
fig, ax = plt.subplots(1, 1)

densities = np.log(layer_sizes) - KS_ARR * np.log(NUM_AAS)
ax.plot(
    KS_ARR, densities, ".-"
)

ax.set_xlabel(f"$k$")
ax.set_ylabel("coverage rate")
ax.set_title("Coverage of all possible $k$-mers")

ax.grid(which='major', color='gray', linestyle='-', alpha=0.6)
ax.grid(which='minor', color='gray', linestyle='--', alpha=0.3)
ax.set_xlim(0, KMAX + 1)

plt.savefig(f"{IMGDIR}/k_vs_coverage_percent.png", bbox_inches=None)
plt.show()

In [ ]:
# Plot total number of kmers, with repeats, as a function of k
fig, ax = plt.subplots(1, 1)

nuniq_kmers = layer_totals

ax.plot(
    KS_ARR, nuniq_kmers, ".-"
)

ax.set_xlabel(f"$k$")
ax.set_ylabel("count")
ax.set_title("Total number of human $k$-mers (with repeats)")

ax.grid(which='major', color='gray', linestyle='-', alpha=0.6)
ax.grid(which='minor', color='gray', linestyle='--', alpha=0.3)
ax.set_xlim(0, ax.get_xlim()[1])

plt.savefig(f"{IMGDIR}/k_vs_total_num_kmers.png", bbox_inches=None)
plt.show()

In [ ]:
nuniq_kmers = layer_totals

# Iterate over the layers of the tree
for i, node_idxs in enumerate(tree.iternodes_in_layers()):
    k = i + 1
    density = densities[i]
    print(f"k={k}")
    #~~~ Plot frequencies of realized k-mers
    counts = tree._get_value_at_node(node_idxs) 

    fig, ax = plt.subplots(1, 1)
    ax.hist(
        counts, 
        bins=30, log=True,
        label=f"total={format(len(node_idxs), ",")}"
    )
    ax.set_xlabel(f"num occurrences in proteome")
    ax.set_ylabel(f"count")
    ax.set_title("Human {}-mer frequencies\n{} unique (log density {:.3g})".format(
        k, format(len(node_idxs), ","), density
    ))
    ax.legend()

    plt.savefig(f"{IMGDIR}/frequencies_{k}mers.png")
    plt.show()


In [ ]:
nuniq_kmers = layer_totals

# Iterate over the layers of the tree
for i, node_idxs in enumerate(tree.iternodes_in_layers()):
    k = i + 1
    density = densities[i]
    print(f"k={k}")
    
    #~~~ Plot number of children
    num_children = tree._get_numchildren_at_node(node_idxs)

    fig, ax = plt.subplots(1, 1)
    ax.hist(
        num_children, 
        bins=30, log=True,
        label=f"total={format(len(node_idxs), ",")}"
    )
    ax.set_xlabel(f"num continuations")
    ax.set_ylabel(f"count")
    ax.set_title("Human {}-mer possible continuations $k\\to k+1$".format(k))

    ax.legend()

    plt.savefig(f"{IMGDIR}/frequencies_{k}mers.png")
    plt.show()


In [ ]:
nuniq_kmers = layer_totals

fig, ax = plt.subplots(1, 1)

ax.set_xlim(0, KMAX + 1)
# Iterate over the layers of the tree
to_plot = [2, 4, 8, 16, 24]
means = np.zeros(KS_ARR.shape)
mins = np.zeros(KS_ARR.shape)
maxs = np.zeros(KS_ARR.shape)
continuations = np.zeros([1 + NUM_AAS, KMAX], dtype=int)
for i, node_idxs in enumerate(tree.iternodes_in_layers()):
    k = i + 1
    density = densities[i]
    print(f"k={k}")
    
    #~~~ Plot number of children
    num_children = tree._get_numchildren_at_node(node_idxs)
    print(num_children.min(), num_children.max(), num_children.mean())

    mins[i] = num_children.min()
    maxs[i] = num_children.max()
    means[i] = num_children.mean()

    from collections import Counter
    counts = Counter(num_children)
    counts = np.array([counts.get(key, 0) for key in range(len(continuations))])
    continuations[:,i] = counts
    
    if k not in to_plot:
        continue
    ax.hist(
        num_children, 
        bins=10, log=True,
        label=f"{k}", alpha=0.2, align="mid",
    )
ax.set_xlabel(f"num continuations")
ax.set_ylabel(f"count")
ax.set_title("Human k-mer possible continuations $k\\to k+1$")

ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="$k$")

plt.savefig(f"{IMGDIR}/frequencies_kmers.png")
plt.show()




In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(KS_ARR, mins, 'r.')
ax.plot(KS_ARR, maxs, 'r.')
ax.plot(KS_ARR, means, 'k-.')
ax.set_xlabel(f"$k$")
ax.set_ylabel(f"mean number of possible continuations")
ax.set_title(f"Average continuation ability")


plt.savefig(f"{IMGDIR}/frequencies_kmers.png")
plt.show()


In [ ]:
fig, ax = plt.subplots(1, 1)

from matplotlib.colors import LogNorm

sc = ax.imshow(
    continuations[:,:-1], 
    cmap='viridis',
    norm=LogNorm(),        # Log normalization
    aspect='equal',        # square cells like matshow
    origin='lower',        # same as matshow default
    interpolation='none',  # no smoothing
)

cbar = fig.colorbar(sc)
cbar.ax.set_ylabel("$k$-mer count")

ax.set_xlabel(f"$k$")
ax.set_ylabel(f"unique continuations available")
ax.set_title("Human k-mer possible continuations $k\\to k+1$")

xts = np.arange(0, KMAX, 2)
yts = np.arange(0, NUM_AAS + 1, 2)
ax.set_xticks(
    xts, labels=[f"{i + 1}" for i in xts]
)
ax.set_yticks(
    yts, labels=[f"{i}" for i in yts]
)


plt.savefig(f"{IMGDIR}/continuation_possibilities.png")
plt.show()

In [ ]:
for x in tree.iternodes_in_layers(max_depth=25):
    print(len(x))

In [ ]:
node_func = lambda nidx: tree._get_numchildren_at_node(nidx)
for depth, x in enumerate(tree.iternodes_in_layers(max_depth=25, node_func=node_func)):
    print(f"Depth {depth}:", np.mean(x), np.median(x), np.min(x), np.max(x))
    

In [ ]:
# Load the sequence data
human_id2seq = {}
for record in SeqIO.parse(human_fasta_fpath, "fasta"):
    seqid = record.id
    seq = sym2int(record.seq, sym2int_map)
    human_id2seq[seqid] = seq

print(f"Loaded {len(human_id2seq)} human protein sequences")